# Error Handling & Session Management Exercise

This exercise demonstrates:
- **Error Handling**: Retry, fallback, graceful degradation strategies
- **Session Management**: File and S3 storage with multi-turn context retention
- **Resilience Testing**: Simulated failures and recovery

## Success Criteria
✅ Agents use session manager for context retention  
✅ Multi-turn conversations maintain full context  
✅ Error handling prevents workflow crashes  
✅ Appropriate strategy for each failure type  
✅ S3 session storage works alongside file storage  

In [1]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands.session import FileSessionManager, S3SessionManager
import json
import time
import random
import os

## Part 1: Tools with Simulated Failures

Three tools prone to different failure types

In [2]:
# Global counters for testing
api_call_count = 0
db_call_count = 0

@tool
def fetch_weather_api(city: str) -> str:
    """
    Fetch weather data - PRONE TO TIMEOUTS.
    Strategy: Retry with exponential backoff
    """
    global api_call_count
    api_call_count += 1
    
    max_retries = 3
    retry_delay = 1
    
    for attempt in range(max_retries):
        try:
            # Simulate 40% timeout rate
            if random.random() < 0.4:
                raise TimeoutError(f"API timeout on attempt {attempt + 1}")
            
            # Success
            result = {
                "city": city,
                "temperature": 72,
                "condition": "Sunny",
                "attempts": attempt + 1
            }
            return json.dumps(result)
        
        except TimeoutError as e:
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                # Final fallback: return cached/default data
                return json.dumps({
                    "city": city,
                    "temperature": "unavailable",
                    "condition": "Data temporarily unavailable",
                    "error": "API timeout after retries",
                    "fallback": True
                })

@tool
def query_customer_database(customer_id: str) -> str:
    """
    Query customer database - PRONE TO CONNECTION ERRORS.
    Strategy: Fallback to cache
    """
    global db_call_count
    db_call_count += 1
    
    try:
        # Simulate 30% connection failure
        if random.random() < 0.3:
            raise ConnectionError("Database connection failed")
        
        # Success
        result = {
            "customer_id": customer_id,
            "name": "John Doe",
            "tier": "Premium",
            "orders": 15,
            "source": "live_database"
        }
        return json.dumps(result)
    
    except ConnectionError:
        # Fallback to cached data
        cached_result = {
            "customer_id": customer_id,
            "name": "John Doe",
            "tier": "Premium",
            "orders": "unknown",
            "source": "cache",
            "warning": "Using cached data - live database unavailable"
        }
        return json.dumps(cached_result)

@tool
def calculate_shipping_cost(weight: float, distance: float) -> str:
    """
    Calculate shipping - PRONE TO VALIDATION ERRORS.
    Strategy: Graceful degradation with estimates
    """
    try:
        # Input validation
        if weight <= 0 or distance <= 0:
            raise ValueError("Weight and distance must be positive")
        
        if weight > 1000:
            raise ValueError("Weight exceeds maximum limit")
        
        # Simulate occasional calculation errors
        if random.random() < 0.2:
            raise RuntimeError("Pricing service temporarily unavailable")
        
        # Success
        cost = (weight * 0.5) + (distance * 0.1)
        result = {
            "weight_lbs": weight,
            "distance_miles": distance,
            "cost": round(cost, 2),
            "accuracy": "exact"
        }
        return json.dumps(result)
    
    except ValueError as e:
        # Return error with guidance
        return json.dumps({
            "error": str(e),
            "cost": None,
            "suggestion": "Please verify input values"
        })
    
    except RuntimeError:
        # Graceful degradation: provide estimate
        estimated_cost = (weight * 0.5) + (distance * 0.1)
        return json.dumps({
            "weight_lbs": weight,
            "distance_miles": distance,
            "cost": round(estimated_cost, 2),
            "accuracy": "estimated",
            "warning": "Using estimated pricing - exact rates unavailable"
        })

## Part 2: Session Manager Setup

### Option 1: File-Based Session Storage

In [26]:
# Initialize file-based session manager
import uuid

session_id = str(uuid.uuid4())
file_session_manager = FileSessionManager(session_id=session_id, base_path="./sessions")

print("✅ File session manager initialized")
print(f"   Sessions stored in: ./sessions")

✅ File session manager initialized
   Sessions stored in: ./sessions


### Option 2: S3-Based Session Storage

In [27]:
# Initialize S3 session manager
# Set your S3 bucket name
S3_BUCKET = os.getenv("SESSION_BUCKET", "my-agent-sessions-bucket-lsakfjhash")
S3_PREFIX = "agent-sessions/"

try:
    s3_session_manager = S3SessionManager(
        session_id=session_id,
        bucket=S3_BUCKET,
        prefix=S3_PREFIX,
        region="us-east-1"
    )
    print("✅ S3 session manager initialized")
    print(f"   Bucket: {S3_BUCKET}")
    print(f"   Prefix: {S3_PREFIX}")
    s3_available = True
except Exception as e:
    print(f"⚠️  S3 session manager not available: {e}")
    print("   Continuing with file-based storage only")
    s3_available = False

✅ S3 session manager initialized
   Bucket: my-agent-sessions-bucket-lsakfjhash
   Prefix: agent-sessions/


## Part 3: Create Resilient Agents

### Agent with File Storage

In [12]:
file_agent = Agent(
    name="file_storage_agent",
    system_prompt="""You are a helpful assistant with access to tools that may occasionally fail.
    
    When tools return errors or warnings:
    - Acknowledge the limitation to the user
    - Use fallback data if available
    - Provide the best answer possible with available information
    - Be transparent about data accuracy (exact vs estimated vs cached)
    
    Remember context from previous turns in the conversation.""",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[fetch_weather_api, query_customer_database, calculate_shipping_cost],
    session_manager=file_session_manager
)

print("✅ File-based agent created")

✅ File-based agent created


### Agent with S3 Storage

In [13]:
if s3_available:
    s3_agent = Agent(
        name="s3_storage_agent",
        system_prompt="""You are a helpful assistant with access to tools that may occasionally fail.
        
        When tools return errors or warnings:
        - Acknowledge the limitation to the user
        - Use fallback data if available
        - Provide the best answer possible with available information
        - Be transparent about data accuracy (exact vs estimated vs cached)
        
        Remember context from previous turns in the conversation.""",
        model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
        tools=[fetch_weather_api, query_customer_database, calculate_shipping_cost],
        session_manager=s3_session_manager
    )
    print("✅ S3-based agent created")
else:
    s3_agent = None
    print("⚠️  S3 agent not available")

✅ S3-based agent created


## Part 4: Test Error Handling Strategies

### Test 1: API Timeout with Retry

In [14]:
print("=== TEST 1: API Timeout with Retry ===")
api_call_count = 0

session_id = "test-session-1"
response = file_agent("What's the weather in Seattle?", session_id=session_id)

print(f"\nResponse: {response}")
print(f"\nAPI calls made: {api_call_count}")
print("✅ Retry logic tested")

=== TEST 1: API Timeout with Retry ===


/Users/emota/miniconda3/envs/oreilly/lib/python3.13/site-packages/strands/agent/agent.py:392: UserWarning: `**kwargs` parameter is deprecating, use `invocation_state` instead.
  async for event in events:


<thinking> To provide the weather information for Seattle, I will use the "fetch_weather_api" tool. However, since this tool is prone to timeouts, I should be prepared to provide an alternative if it fails.</thinking>

Tool #1: fetch_weather_api
The weather in Seattle today is sunny with a temperature of 72 degrees Fahrenheit. This information was fetched directly from the weather API. If you need any more details or have another request, feel free to ask!
Response: The weather in Seattle today is sunny with a temperature of 72 degrees Fahrenheit. This information was fetched directly from the weather API. If you need any more details or have another request, feel free to ask!


API calls made: 1
✅ Retry logic tested


### Test 2: Database Connection Failure with Fallback

In [15]:
print("=== TEST 2: Database Failure with Fallback ===")
db_call_count = 0

response = file_agent("Look up customer CUST-12345", session_id=session_id)

print(f"\nResponse: {response}")
print(f"\nDatabase calls made: {db_call_count}")
print("✅ Fallback to cache tested")

=== TEST 2: Database Failure with Fallback ===
<thinking> To provide information about the customer with ID "CUST-12345", I will use the "query_customer_database" tool. Since this tool is prone to connection errors, I will also consider falling back to cached data if necessary.</thinking> 
Tool #2: query_customer_database
Here is the information for customer CUST-12345:
- Name: John Doe
- Customer Tier: Premium
- Number of Orders: 15

This information was retrieved from the live customer database. If you need any more details or have another request, feel free to ask!
Response: Here is the information for customer CUST-12345:
- Name: John Doe
- Customer Tier: Premium
- Number of Orders: 15

This information was retrieved from the live customer database. If you need any more details or have another request, feel free to ask!


Database calls made: 1
✅ Fallback to cache tested


### Test 3: Validation Error with Graceful Degradation

In [16]:
print("=== TEST 3: Validation Error ===")

response = file_agent(
    "Calculate shipping cost for 25 lbs package going 500 miles",
    session_id=session_id
)

print(f"\nResponse: {response}")
print("✅ Graceful degradation tested")

=== TEST 3: Validation Error ===
<thinking> To calculate the shipping cost for a 25 lbs package traveling 500 miles, I will use the "calculate_shipping_cost" tool. This tool is prone to validation errors, so I will provide an estimated cost in case it fails.</thinking> 
Tool #3: calculate_shipping_cost


If there are any errors, I will inform you that I cannot provide an exact cost but can estimate based on typical rates.The exact shipping cost for a 25 lbs package traveling 500 miles is $62.50. This calculation is based on the provided shipping tool. If you need further assistance or have another request, just let me know!
Response: The exact shipping cost for a 25 lbs package traveling 500 miles is $62.50. This calculation is based on the provided shipping tool. If you need further assistance or have another request, just let me know!

✅ Graceful degradation tested


## Part 5: Test Session Management

### Multi-Turn Conversation with File Storage

In [17]:
print("=== MULTI-TURN CONVERSATION (File Storage) ===")

session_id = "test-session-2"

# Turn 1: Establish context
print("\n--- Turn 1 ---")
response1 = file_agent(
    "What's the weather in Portland?",
    session_id=session_id
)
print(f"Agent: {response1}")

# Turn 2: Reference previous context
print("\n--- Turn 2 ---")
response2 = file_agent(
    "How does that compare to Seattle?",
    session_id=session_id
)
print(f"Agent: {response2}")

# Turn 3: Continue building context
print("\n--- Turn 3 ---")
response3 = file_agent(
    "Which city should I visit based on the weather?",
    session_id=session_id
)
print(f"Agent: {response3}")

print("\n✅ File storage context retention verified")

=== MULTI-TURN CONVERSATION (File Storage) ===

--- Turn 1 ---
<thinking> To provide the weather information for Portland, I will use the "fetch_weather_api" tool again. Since this tool is prone to timeouts, I should be prepared to provide an alternative if it fails.</thinking> 
Tool #4: fetch_weather_api


If the tool fails, I will inform you that I cannot provide the exact weather but can offer an estimated forecast based on typical weather patterns in Portland.The weather in Portland today is sunny with a temperature of 72 degrees Fahrenheit. This information was fetched from the weather API after two attempts. If you need any more details or have another request, feel free to ask!Agent: The weather in Portland today is sunny with a temperature of 72 degrees Fahrenheit. This information was fetched from the weather API after two attempts. If you need any more details or have another request, feel free to ask!


--- Turn 2 ---
<thinking> To compare the weather between Portland and Se

### Multi-Turn Conversation with S3 Storage

In [18]:
if s3_available:
    print("=== MULTI-TURN CONVERSATION (S3 Storage) ===")
    
    session_id = "test-session-s3"
    
    # Turn 1: Establish context
    print("\n--- Turn 1 ---")
    response1 = s3_agent(
        "What's the weather in Boston?",
        session_id=session_id
    )
    print(f"Agent: {response1}")
    
    # Turn 2: Reference previous context
    print("\n--- Turn 2 ---")
    response2 = s3_agent(
        "How does that compare to New York?",
        session_id=session_id
    )
    print(f"Agent: {response2}")
    
    # Turn 3: Continue building context
    print("\n--- Turn 3 ---")
    response3 = s3_agent(
        "Which city has better weather?",
        session_id=session_id
    )
    print(f"Agent: {response3}")
    
    print("\n✅ S3 storage context retention verified")
else:
    print("⚠️  Skipping S3 multi-turn test (S3 not available)")

=== MULTI-TURN CONVERSATION (S3 Storage) ===

--- Turn 1 ---
<thinking>To provide the user with the weather information for Boston, I need to use the "fetch_weather_api" tool. Given that this tool is prone to timeouts, I will proceed with using it.</thinking>

Tool #1: fetch_weather_api
<thinking>The "fetch_weather_api" tool has successfully returned the weather information for Boston. The data includes the temperature and weather condition. Given that the attempts count is 1, it indicates that the tool did not face any significant issues fetching the data.</thinking> 

The weather in Boston is currently 72°F and sunny.Agent: <thinking>The "fetch_weather_api" tool has successfully returned the weather information for Boston. The data includes the temperature and weather condition. Given that the attempts count is 1, it indicates that the tool did not face any significant issues fetching the data.</thinking> 

The weather in Boston is currently 72°F and sunny.


--- Turn 2 ---
<thinking

## Part 6: Compare Storage Backends

In [19]:
print("=== STORAGE BACKEND COMPARISON ===")
print()
print("File Storage:")
print("  ✅ Fast local access")
print("  ✅ No external dependencies")
print("  ✅ Good for development/testing")
print("  ⚠️  Not suitable for distributed systems")
print("  ⚠️  No automatic backup")
print()
print("S3 Storage:")
print("  ✅ Durable and highly available")
print("  ✅ Scales automatically")
print("  ✅ Shared across multiple agents/instances")
print("  ✅ Built-in versioning and backup")
print("  ⚠️  Slightly higher latency")
print("  ⚠️  Requires AWS credentials")

=== STORAGE BACKEND COMPARISON ===

File Storage:
  ✅ Fast local access
  ✅ No external dependencies
  ✅ Good for development/testing
  ⚠️  Not suitable for distributed systems
  ⚠️  No automatic backup

S3 Storage:
  ✅ Durable and highly available
  ✅ Scales automatically
  ✅ Shared across multiple agents/instances
  ✅ Built-in versioning and backup
  ⚠️  Slightly higher latency
  ⚠️  Requires AWS credentials


## Key Learnings

### Error Handling Strategies
1. **Retry with Exponential Backoff**: For transient failures (timeouts, rate limits)
2. **Fallback to Cache**: For service unavailability
3. **Graceful Degradation**: Provide estimates when exact data unavailable
4. **Input Validation**: Catch errors early with clear messages

### Session Management
1. **FileSessionManager**: Fast local storage for development
2. **S3SessionManager**: Durable, scalable storage for production
3. **Context Retention**: Multi-turn conversations maintain history
4. **Session Persistence**: Survives agent restarts

### Storage Backend Selection
- **Development/Testing**: Use FileSessionManager
- **Production/Distributed**: Use S3SessionManager
- **Hybrid**: File for local dev, S3 for deployed environments

### Resilience Testing
1. **Simulated Failures**: Random failures test real-world scenarios
2. **Multiple Failure Types**: Timeout, connection, validation errors
3. **Recovery Verification**: Agent continues functioning after errors